In [1]:
import sympy
from sympy import Derivative, Function, Symbol, symbols, Eq, pi, cos, sin, exp, log, oo
from sympy import Function, dsolve, Derivative, checkodesol
from sympy import fps, Rational
from sympy import pprint, Matrix, eye, zeros
from sympy import Inverse
from utils import *
z = Symbol('z')

Restate the problem that we want to solve: 

$$z^k \frac{d\psi}{dz} = A(z)\psi$$, where $\psi$ is a vector-valued function and $A(z)$ as a given matrix-valued function is holomorphic.

In the general case where we can't find it directly, we use Formal Gauge transform to simplify the system, and solve it in diagonal version. Then, $\hat{\Phi}(u, z) = \hat{\Psi}(t(u, z)) \hat{\Psi}^{-1}(s(u,z))$ is the formal universal solution on Sto_k, while since $\Phi$ is unique, then this means that $\hat{\Phi}$ is the power series of $\Phi$, so it must be convergent since $\Phi$ is entire.

To do the Gauge transformation, the core idea is to transfer it to the case where $z^k \frac{d\phi}{dz} = (D_0+zD_1+...z^{k-1}D_{k-1})\phi$, where $D_i$ are diagonal, which can be easily solve by entries on both sides.

In order to solve this problem, we first need to make a change of variables $\psi' = F \psi$, whereas $F = 1+zF_1 + z^2F_2 + ...$









In this step, we let $F = (I+zH_1)(I+z^2H_2)...$

where $H_p = \begin{cases} {ad}^{-1}_{A_0}(A_p^{OD}) \text{   if   } k > 1 \\ ({ad}_{A_0} - p)^{-1}(A_p^{OD}) \text{    if   } k = 1  \end{cases}$

After this transform, the ODE becomes $z^k \frac{d\psi'}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...) \psi'$

After doing this, we make the second Gauge Transform to transform it to the finite case, 

which is the transform $K = \exp(-\int{D_k+D_{k+1}z+...})$


Finally, the simplified system is $z^k \frac{d\psi''}{dz} = (diagonal(A_0) + diagonal(A_1)z + ...+ diagonal(A_{k-1})z^{k-1}) \psi''$

and the overall Gauge transform is $(KF)$[...], and the inverse is $(KF)^{-1}$[...], so $\psi = (KF)^{-1}\psi''$

After we get the fundamental solution $\Psi$, we can construct $\Phi$ on the Sto_k in a similar way.

However, in order for the above approach to work, we have to impose the condition that the ODE is generic.

This means that the leading term $A(0)$ is diagonalizable, with eigenvalues $(\alpha_1, ..., \alpha_n)$ satisfying that 

1. if $k \geq 2$, then they are distinct 

2. if $k = 1$, they no pair of eigenvalues differs by a positive integer.

This is known as the non-resonance assumption.

Example 1: 

$z \psi' = \begin{pmatrix} \frac{1}{2} & z \\ 0 & 1 \end{pmatrix} \psi$ 

We are working on Sto_1

We trancate the Gauge Transform series up to some given order 

Since this matrix is upper triangular, it can be solved directly:

Firstly solve $\psi_2$, which is $Az$, 

then substitute this into the equation of $\psi_1$ and get $z\psi'_1 = \frac{1}{2}\psi_1 + Az^2$, then we get

$\psi_1 = \frac{2A}{3}z^2 + B \sqrt{z}$, where $A$, $B$ are constants

so $\psi = \begin{pmatrix} \frac{2A}{3}z^2 + B \sqrt{z} \\ Az \end{pmatrix} = A\begin{pmatrix} \frac{2}{3}z^2 \\ z \end{pmatrix} + B\begin{pmatrix} \sqrt{z} \\ 0 \end{pmatrix}$

In our method, we get $F = (I + \begin{pmatrix} 0 & -\frac{2}{3} \\ 0 & 0 \end{pmatrix}z)$

In [2]:
A = Matrix([[Rational(1,2), z], [0, 1]])
n = 2
k = 1
order = 5

In [3]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [6]:
Total

Matrix([
[1, -2*z/3],
[0,      1]])

In [7]:
Total_inverse = Total.inverse()

In [8]:
Total_inverse

Matrix([
[1, 2*z/3],
[0,     1]])

In [9]:
Psi_prime, Phi_prime = solve_phi(k, A)

In [10]:
Psi_prime

Matrix([
[sqrt(z), 0],
[      0, z]])

In [11]:
Psi = Total_inverse * Psi_prime
Psi = Psi.simplify()

In [12]:
Psi

Matrix([
[sqrt(z), 2*z**2/3],
[      0,        z]])

This value of $\Psi$ matches exactly with our manually found solution stated in the beginning!!

In [13]:
Final_Phi = get_phi_from_psi(Psi, k)

In [14]:
Final_Phi

Matrix([
[sqrt(z*exp(u))/sqrt(z), -2*sqrt(z)*sqrt(z*exp(u))/3 + 2*z*exp(2*u)/3],
[                     0,                                       exp(u)]])

This is our final universal solution in the Stokes groupoid

Example2: 

$z^2 \psi' = \begin{pmatrix} \cos(z)+1 & \sin(z) \\ -\sin(z) & \cos(z) \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [15]:
A = Matrix([[cos(z)+1, sin(z)], [-sin(z), cos(z)]])
n = 2
k = 2
order = 3
# This means that we only approaching using the first 3 terms

In [16]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [17]:
K_trunc

Matrix([
[exp(z/2),        0],
[       0, exp(z/2)]])

In [18]:
F_trunc

Matrix([
[1, z],
[z, 1]])

In [19]:
Total

Matrix([
[  exp(z/2), z*exp(z/2)],
[z*exp(z/2),   exp(z/2)]])

In [20]:
Total_inverse = Total.inverse()

In [21]:
Total_inverse

Matrix([
[exp(z)/(-z**2*exp(3*z/2) + exp(3*z/2)), z*exp(z)/(z**2*exp(3*z/2) - exp(3*z/2))],
[     z*exp(z/2)/(z**2*exp(z) - exp(z)),        exp(z/2)/(-z**2*exp(z) + exp(z))]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [22]:
Psi_prime, Phi_prime = solve_phi(k, A)

In [23]:
Psi_prime

Matrix([
[exp(-2/z),         0],
[        0, exp(-1/z)]])

In [24]:
Phi_prime

Matrix([
[exp(2*(1 - exp(-u*z))/z),                      0],
[                       0, exp((1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the initial change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

So the final solution $\Psi$ is 

In [25]:
Psi = Total_inverse * Psi_prime

Hence, the original universal solution in the Stokes groupoid of order $k$ is 

In [26]:
Phi = get_phi_from_psi(Psi, k)

In [27]:
Phi

Matrix([
[(z**2*exp(u*z + exp(-u*z)/z) - exp(1/z))*exp(-z*exp(u*z)/2 + z/2 + 1/z - 2*exp(-u*z)/z)/(z**2*exp(2*u*z) - 1),   z*(-exp(1/z) + exp(u*z + exp(-u*z)/z))*exp(-z*exp(u*z)/2 + z/2 + 1/z - 2*exp(-u*z)/z)/(z**2*exp(2*u*z) - 1)],
[  z*(-exp(exp(-u*z)/z) + exp(u*z + 1/z))*exp(-z*exp(u*z)/2 + z/2 + 1/z - 2*exp(-u*z)/z)/(z**2*exp(2*u*z) - 1), (z**2*exp(u*z + 1/z) - exp(exp(-u*z)/z))*exp(-z*exp(u*z)/2 + z/2 + 1/z - 2*exp(-u*z)/z)/(z**2*exp(2*u*z) - 1)]])

Example2: 

$z^2 \psi' = \begin{pmatrix} z^4+3z^3+1 & z \\ z^2 & z^3+5z^2+7 \end{pmatrix} \psi$ 

We are working on Sto_2

We trancate the Gauge Transform series up to some given order 

In [28]:
A = Matrix([[z**4+3*z**3+1, z], [z**2, z**3+5*z**2+7]])
n = 2
k = 2
order = 10

In [29]:
K_trunc, F_trunc, Total = get_Gauge_up_to_order(A, k, order)

In [30]:
Total

Matrix([
[(36 - z**3)*exp(-z**2*(2*z + 9)/6)/36, -z*exp(-z**2*(2*z + 9)/6)/6],
[            z**2*exp(-z*(z + 10)/2)/6,          exp(-z*(z + 10)/2)]])

In [31]:
Total_inverse = Total.inverse()

In [32]:
Total_inverse

Matrix([
[                     exp(-z**3/3 - 3*z**2/2)*exp(2*z**3/3 + 3*z**2),                                                                                                                        z*exp(z**2/2 + 5*z)/6],
[-z**2*exp(-z**2/2 - 5*z)*exp(z**2/2 + 5*z)*exp(z**3/3 + 3*z**2/2)/6, -z**3*exp(z**2/2 + 5*z)*exp(-z**3/3 - 3*z**2/2)*exp(z**3/3 + 3*z**2/2)/36 + exp(z**2/2 + 5*z)*exp(-z**3/3 - 3*z**2/2)*exp(z**3/3 + 3*z**2/2)]])

Note that Gauge Transform is a group action, meaning that $F_1[F_2[A]] = (F_1F_2)[A]$ so the inverse Gauge Transform is the Gauge Transform of the inverse Matrix, and composed Gauge Transform is the transform of the product Matrix.

Finally, the equation is $z^k \psi' = diag(A_0) + diag(A_1)z + ... + diag(A_{k-1})z^{k-1}$

In [33]:
Psi_prime, Phi_prime = solve_phi(k, A)

In [34]:
Psi_prime

Matrix([
[exp(-1/z),         0],
[        0, exp(-7/z)]])

In [35]:
Phi_prime

Matrix([
[exp((1 - exp(-u*z))/z),                        0],
[                     0, exp(7*(1 - exp(-u*z))/z)]])

The final solution is $T^{-1}\Psi$ (we use $T$ to denote the change of coordinate that we applied to simplify the system)

$T^{-1}$ is the Total_inverse that we just found above.

In [36]:
T_inverse_Psi = Total_inverse * Psi_prime

In [37]:
T_inverse_Psi

Matrix([
[                     exp(-1/z)*exp(-z**3/3 - 3*z**2/2)*exp(2*z**3/3 + 3*z**2),                                                                                                                          z*exp(-7/z)*exp(z**2/2 + 5*z)/6],
[-z**2*exp(-1/z)*exp(-z**2/2 - 5*z)*exp(z**2/2 + 5*z)*exp(z**3/3 + 3*z**2/2)/6, (-z**3*exp(z**2/2 + 5*z)*exp(-z**3/3 - 3*z**2/2)*exp(z**3/3 + 3*z**2/2)/36 + exp(z**2/2 + 5*z)*exp(-z**3/3 - 3*z**2/2)*exp(z**3/3 + 3*z**2/2))*exp(-7/z)]])

In [38]:
Final_Phi = get_phi_from_psi(T_inverse_Psi, k)
## This will take much long time to run.

In [39]:
Final_Phi

Matrix([
[                                                                    (-z**3*exp(z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 6*exp(-u*z)/z) + z**3*exp(u*z + z**3/3 + z**2 + 5*z*exp(u*z) + 6/z) + 36*exp(z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 6*exp(-u*z)/z))*exp(-z**3/3 + z**2*exp(2*u*z)/2 - 3*z**2/2 - 5*z + 1/z - 7*exp(-u*z)/z)/36,                                                                z*(-exp(z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 6*exp(-u*z)/z) + exp(u*z + z**3/3 + z**2 + 5*z*exp(u*z) + 6/z))*exp(-z**3/3 + z**2*exp(2*u*z)/2 - 3*z**2/2 - 5*z + 1/z - 7*exp(-u*z)/z)/6],
[z**2*(z**3*exp(2*u*z + z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 6*exp(-u*z)/z) - z**3*exp(3*u*z + z**3/3 + z**2 + 5*z*exp(u*z) + 6/z) + 36*exp(z**3/3 + z**2 + 5*z*exp(u*z) + 6/z) - 36*exp(2*u*z + z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 6*exp(-u*z)/z))*exp(-z**3/3 + z**2*exp(2*u*z)/2 - 3*z**2/2 - 5*z + 1/z - 7*exp(-u*z)/z)/216, (z**3*exp(2*u*z + z**3*exp(3*u*z)/3 + z**2*exp(2*u*z) + 5*z + 

Even though we could Finally get $\Phi$ of the original system, which is clearly smooth, a limitation that we could optimize is its running time, yet nevertheless the computational ability has already surpassed human being.